## Import relevant Python Modules

In [1]:
import geopandas as gpd
import contextily as cx
import matplotlib as plt
import shapely
import cartopy.crs as ccris
import os

## Import data

In [3]:
substations = gpd.read_file(os.path.abspath('Data/belfastSubstations.shp'))
EVcharging = gpd.read_file(os.path.abspath('Data/evChargingStations_belfast.shp'))

## Check CRS

In [4]:
substations.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Convert CRS to match basemap

In [5]:
substations_wgs = substations.to_crs(epsg=3857)
EVcharging_wgs = EVcharging.to_crs(epsg=3857)

## Create Plot